# Project 4 Notebook: NLP Classification

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re

In [6]:
#Creation of Dataframe

tweets = pd.read_csv('./data/tweets.csv', encoding='unicode_escape')
tweets.shape

(9093, 3)

In [7]:
tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [8]:
tweets.loc[0].tweet_text

'.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.'

In [12]:
#Creation of Dictionary containing Classification

classif_dict = {'Classification': ['Positive', 'Negative'], 'Numeric': [0, 1]}

In [15]:
#Creation of predictied classification column

tweets['predicted_classif'] = [[] for x in tweets['tweet_text']]

In [16]:
tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,predicted_classif
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,[]
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,[]
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,[]
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,[]
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,[]


## Data Cleaning

In [17]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [18]:
tweets['emotion_in_tweet_is_directed_at'].fillna('None', inplace=True)

In [19]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     9093 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [20]:
tweets['emotion_in_tweet_is_directed_at'].replace('None', 'Unknown', inplace = True)

In [21]:
tweets.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,predicted_classif
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,[]
9089,"Wave, buzz... RT @mention We interrupt your re...",Unknown,No emotion toward brand or product,[]
9090,"Google's Zeiger, a physician never reported po...",Unknown,No emotion toward brand or product,[]
9091,Some Verizon iPhone customers complained their...,Unknown,No emotion toward brand or product,[]
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,Unknown,No emotion toward brand or product,[]


In [22]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     9093 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [23]:
tweets['tweet_text'].fillna('-', inplace = True)

In [24]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9093 non-null   object
 1   emotion_in_tweet_is_directed_at                     9093 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB
